In [17]:
import sys
!{sys.executable} -m pip install psycopg2-binary
!{sys.executable} -m pip install ipynb
!{sys.executable} -m pip install pyyaml
import psycopg2

     |████████████████████████████████| 249 kB 561 kB/s eta 0:00:01


In [24]:
import os
from threading import Thread
from flask import Flask, request, jsonify
from datetime import datetime
import json
import yaml
import numpy as np
import pandas.io.sql as sqlio
import scipy
import numpy.linalg as LA
from sklearn import preprocessing, decomposition
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import cosine
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import nltk
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from collections import OrderedDict, defaultdict
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

from ipynb.fs.full.training_and_clustering import *
config = yaml.load(open("config.yml"), Loader=yaml.FullLoader)

In [25]:
app = Flask(__name__)
config = yaml.safe_load(open('config.yml', 'r'))
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_HOME"] = config['base_model_dir']

In [21]:
@app.route("/extract_intents", methods=["POST", "GET"])
def extract_intents():
    org_id = request.args['org_id']
    threshold = request.args['threshold']

    from datasets import load_dataset
    train_dataset = load_dataset("swda", split='train')
    test_dataset = load_dataset("swda", split='test')

    dimensions = set()
    train_utterance_tags, test_utterance_tags = {}, {}
    for i in range(len(train_dataset)):
        if i > 60000:
            break
        utterance = train_dataset[i]['text']
        tag = train_dataset[i]['topic_description']
        dimensions.add(tag)
        train_utterance_tags[utterance] = tag

    for i in range(len(test_dataset)):
        utterance = test_dataset[i]['text']
        tag = test_dataset[i]['topic_description']
        dimensions.add(tag)
        test_utterance_tags[utterance] = tag

    print("Testing...")

    model = org_model_map[org_id]
    clusters, ids2prems, ids2embeds, ids2projection_embeds = embed_and_cluster(train_utterance_tags, test_utterance_tags,
                                                        model, config)

    with open('./data_files/misc/ids2prems.pickle', 'rb') as f:
        ids2prems = pickle.load(f)

    with open('./data_files/misc/ids2embeds.pickle', 'rb') as f:
        ids2embeds = pickle.load(f)

    with open('./data_files/misc/ids2projection_embeds.pickle', 'rb') as f:
        ids2projection_embeds = pickle.load(f)

    with open('./data_files/misc/clusters.pickle', 'rb') as f:
        clusters = pickle.load(f)

    tars = load_tars_classifier_miscellaneous(config)
    tagged_clusters_output = tag_dimension_to_clusters_miscellaneous(tars, clusters, ids2prems, ids2embeds,
                                                                     ids2projection_embeds, float(threshold))

    return jsonify(tagged_clusters_output)

In [22]:
if __name__ == '__main__':
    app.run(host="0.0.0.0", port="8000", debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
